# TESS LC BATCH FILE PROCESSING

In [1]:
# NOTA IMPORTANTE - CÓMO IMPLEMENTAR LO DEL "TIMEOUT".
# https://stackoverflow.com/questions/25027122/break-the-function-after-certain-time
# NO SIRVE - SOLO SIRVE PARA ENTORNOS UNIX

This notebook takes the GTO objects table and calculates the Lomb-Scargle periodogram for each of the available TESS light curves, storing the results in `FITS` files with a primary header and two additional header data units, one with the light curve itself and another with the periodogram.

Additionally, the GTO objects table and file is updated with new columns storing the results for each object.

Error control is implemented to prevent exceptions and timeouts to interrupt the processing. In case any of these errors happen, it is so recorded in the results table. Timeout value is dinamically updated, as the calculation for each record is completed.

**Calculation conditions:**

The stars currently under analysis are in the $He_{3}$ instability band, so they are expected to have effective temperatures in the range $T_{eff}\in[3300, 4300] K$, $\log g\in[4.5, 5.1]$, and masses in the range $M_{star}\in[0.20, 0.60] M_{Sun}$.

According to _Table 2_ in _The theoretical instability strip of M dwarf stars (Rodríguez-López, C., et al. 2014, MNRAS, 438, 2371)_ these stars have typical periods of $20 min$ to $3 h$, so we will set the limits of the periodogram frequencies for periods between $10 min$ and $5 h$. In days, this corresponds to a range of $P\in[0.006944, 0.208333] d$ or, equivalently, frequencies in the range $f\in[144, 4.8] d^{-1}$.

We will use the _Generalized Lomb Scargle Periodogram_ method, as described in [Zechmeister and Kürster, 2009](https://www.aanda.org/articles/aa/full_html/2009/11/aa11296-08/aa11296-08.html) and implemented by GitHub repository [mzechmeister/GLS](https://github.com/mzechmeister/GLS), using the default _ZK_ normalization.

## Modules and configuration

### Modules

In [2]:
# Modules import:
#from collections import OrderedDict
import pandas as pd
import numpy as np
import time

#from scipy import stats

# https://github.com/mzechmeister/GLS
from gls import Gls

from astropy.table import Table, QTable
#from astropy.timeseries import TimeSeries
from astropy import units as u
from astropy.io import fits
#from astropy.time import Time

import lightkurve as lk

#%matplotlib inline
import matplotlib.pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 11, 11

#import seaborn as sns
#sns.set_style("white", {'figure.figsize':(15,10)})
#sns.set_style("whitegrid")
#sns.set(rc={'figure.figsize':(15,8)})

### Configuration

In [3]:
# Configuration:
# Files and folders:
GTO_FILE = "../data/GTO_objects_with_PG.csv"
IN_LC_FOLDER = "../data/CARMENES_GTO_TESS_lc/lightcurves/"
OUT_IMG_FOLDER = "../data/CARMENES_GTO_TESS_PGs/figures/"
OUT_PROCESSED_FOLDER = "../data/CARMENES_GTO_TESS_PGs/"
#OUT_GTO_FILE = "../data/GTO_objects_withRVPG.csv"
#OUT_PG_FILE = "../data/GTO_PGs.csv"
#OUT_ERROR_FILE = "../data/GTO_PG_ERRORs.csv"

# Periodogram constants:
FBEG = 4.8
FEND = 144
PBEG = None # Default
PEND = None # Default
OFAC = 10 # Default
HIFAC = 1 # Default
FREQ = np.linspace(start=FBEG, stop=FEND, num=1000001) # 1,000,001 points between 4.8 and 144 d^(-1)
# Must be compatible with FBEG, FEND values
NORM = "ZK" # Default
LS = False # Default
FAST = False # Default
VERBOSE = False # Default
FAP_LEVELS_PLOT = [0.01, 0.05, 0.10] # FAP reference levels to plot

# Initial timeout value (to prevent a stuck file to interrupt the whole process)
#INITIAL_TIMEOUT = 600 # Seconds


In [4]:
def tess_lc_load(filename: str):
    '''Load the TESS LC file and returns a lightcurve object with just the three relevant columns
    Note: it seems to be necessary to drop 'nan' values for GLS to work properly.'''
    lc_lk = lk.read(filename).remove_nans()
    return lc_lk

In [5]:
def lc_infer_sampling(lc_lk: lk.LightCurve):
    '''Infer sampling period from light curve'''
    time_diffs = lc_lk['time'][1:] - lc_lk['time'][:-1]
    return np.median(time_diffs)

## Data processing

### GTO data loading

In [6]:
# Load GTO data table:
gto = pd.read_csv(GTO_FILE, sep=',', decimal='.')
gto.head(5)

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,T0_PG_TESS,e_T0_PG_TESS,offset_PG_TESS,e_offset_PG_TESS,FAP_PG_TESS,valid_PG_TESS,error_PG_TESS,elapsed_time_PG_TESS,fits_file_TESS,fig_file_TESS
0,J23585+076,Wolf 1051,AB,4383,359.636420,7.656947,23:58:32.65,+07:39:30.1,100.839686,-52.931049,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.002199,NaN,NaN
1,J23556-061,GJ 912,AB,912,358.913617,-6.144283,23:55:39.78,-06:08:33.4,88.129933,-65.175491,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000974,NaN,NaN
2,J23548+385,RX J2354.8+3831,-,NaN,358.713658,38.526340,23:54:51.46,+38:31:36.2,110.941908,-23.024449,...,1764.609498,0.010704,8262.219751,1.365667,1.000000,1.0,NaN,344.002685,../data/CARMENES_GTO_TESS_PGs/J23548+385_TESS_...,../data/CARMENES_GTO_TESS_PGs/figures/J23548+3...
3,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,1354.108815,0.001261,7767.134654,0.094298,0.064148,1.0,NaN,473.533042,../data/CARMENES_GTO_TESS_PGs/J23505-095_TESS_...,../data/CARMENES_GTO_TESS_PGs/figures/J23505-0...
4,J23492+024,BR Psc,-,908,357.306604,2.396918,23:49:12.53,+02:24:04.4,93.567467,-56.885396,...,NaN,NaN,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000984,NaN,NaN


In [7]:
print(list(gto.columns))

['Karmn', 'Name', 'Comp', 'GJ', 'RA_J2016_deg', 'DE_J2016_deg', 'RA_J2000', 'DE_J2000', 'l_J2016_deg', 'b_J2016_deg', 'Ref01', 'SpT', 'SpTnum', 'Ref02', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'Ref03', 'L_Lsol', 'eL_Lsol', 'Ref04', 'R_Rsol', 'eR_Rsol', 'Ref05', 'M_Msol', 'eM_Msol', 'Ref06', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'Ref07', 'pi_mas', 'epi_mas', 'Ref08', 'd_pc', 'ed_pc', 'Ref09', 'Vr_kms-1', 'eVr_kms-1', 'Ref10', 'ruwe', 'Ref11', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1', 'W_kms-1', 'eW_kms-1', 'Ref12', 'sa_m/s/a', 'esa_m/s/a', 'Ref13', 'SKG', 'Ref14', 'SKG_lit', 'Ref14_lit', 'Pop', 'Ref15', 'vsini_flag', 'vsini_kms-1', 'evsini_kms-1', 'Ref16', 'P_d', 'eP_d', 'Ref17', 'pEWHalpha_A', 'epEWHalpha_A', 'Ref18', 'log(LHalpha/Lbol)', 'elog(LHalpha/Lbol)', 'Ref19', '1RXS', 'CRT_s-1', 'eCRT_s-1', 'HR1', 'eHR1', 'HR2', 'eHR2', 'Flux_X_E-13_ergcm-2s-1', 'eFlux_X_E-13_ergcm-2s-1', 'LX/LJ', 'eLX/LJ', 'Ref20', 'Activity', 'Ref21', 'FUV_mag',

In [8]:
gto.loc[72, ['Karmn', 'has_lc', 'lc_file']]

Karmn                                             J18419+318
has_lc                                                  True
lc_file    ../data/CARMENES_GTO_TESS_lc/lightcurves/J1841...
Name: 72, dtype: object

Generate the proper auxiliary columns (with the basic periodograms results).

In [9]:
# Additional columns:
if 'n_TESS' in gto.columns:
    pass
else:
    gto['n_TESS'] = None # Number of points in TESS LC
    gto['Ps_TESS'] = None # Sampling period (d)
    gto['fs_TESS'] = None # Sampling frequency (d^(-1))
    gto['wmean_TESS'] = None # Mean of TESS LC
    gto['wrms_TESS'] = None # RMS of TESS LC
    gto['info_PG_TESS'] = None # Information text about the PG
    gto['maxP_PG_TESS'] = None # Max power value in the PG
    gto['maxSNR_PG_TESS'] = None # Max power value in the PG
    gto['rms_PG_TESS'] = None # RMS value in the PG residuals
    gto['f_PG_TESS'] = None # Best frequency in the PG (d^(-1))
    gto['e_f_PG_TESS'] = None # Error of the best frequency in the PG (d^(-1))
    gto['Pd_PG_TESS'] = None # Best period in the PG (d)
    gto['e_Pd_PG_TESS'] = None # Error of the best period in the PG (d)
    gto['Ph_PG_TESS'] = None # Best period in the PG (hours)
    gto['e_Ph_PG_TESS'] = None # Error of the best period in the PG (hours)
    gto['Pm_PG_TESS'] = None # Best period in the PG (minutes)
    gto['e_Pm_PG_TESS'] = None # Error of the best period in the PG (minutes)
    gto['A_PG_TESS'] = None # Amplitude of the best frequency
    gto['e_A_PG_TESS'] = None # Error of the amplitude of the best frequency
    gto['ph_PG_TESS'] = None # Amplitude of the best frequency
    gto['e_ph_PG_TESS'] = None # Error of the amplitude of the best frequency
    gto['T0_PG_TESS'] = None # Reference epoch of the best frequency
    gto['e_T0_PG_TESS'] = None # Error of the epoch of the best frequency
    gto['offset_PG_TESS'] = None # Offset of the best frequency
    gto['e_offset_PG_TESS'] = None # Error of the offset of the best frequency
    gto['FAP_PG_TESS'] = None # False alarm probability
    gto['valid_PG_TESS'] = None # Flag to indicate if the periodogram calculation succeeded (1) or not (0).
    gto['error_PG_TESS'] = None # The error raised during processing. Empty if processing was successful.
    gto['elapsed_time_PG_TESS'] = None # The time elapsed in calculation
    gto['fits_file_TESS'] = None # The name of the processed file.
    gto['fig_file_TESS'] = None # The name of the figure file.


In [10]:
print(list(gto.columns))

['Karmn', 'Name', 'Comp', 'GJ', 'RA_J2016_deg', 'DE_J2016_deg', 'RA_J2000', 'DE_J2000', 'l_J2016_deg', 'b_J2016_deg', 'Ref01', 'SpT', 'SpTnum', 'Ref02', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'Ref03', 'L_Lsol', 'eL_Lsol', 'Ref04', 'R_Rsol', 'eR_Rsol', 'Ref05', 'M_Msol', 'eM_Msol', 'Ref06', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'Ref07', 'pi_mas', 'epi_mas', 'Ref08', 'd_pc', 'ed_pc', 'Ref09', 'Vr_kms-1', 'eVr_kms-1', 'Ref10', 'ruwe', 'Ref11', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1', 'W_kms-1', 'eW_kms-1', 'Ref12', 'sa_m/s/a', 'esa_m/s/a', 'Ref13', 'SKG', 'Ref14', 'SKG_lit', 'Ref14_lit', 'Pop', 'Ref15', 'vsini_flag', 'vsini_kms-1', 'evsini_kms-1', 'Ref16', 'P_d', 'eP_d', 'Ref17', 'pEWHalpha_A', 'epEWHalpha_A', 'Ref18', 'log(LHalpha/Lbol)', 'elog(LHalpha/Lbol)', 'Ref19', '1RXS', 'CRT_s-1', 'eCRT_s-1', 'HR1', 'eHR1', 'HR2', 'eHR2', 'Flux_X_E-13_ergcm-2s-1', 'eFlux_X_E-13_ergcm-2s-1', 'LX/LJ', 'eLX/LJ', 'Ref20', 'Activity', 'Ref21', 'FUV_mag',

### Batch processing of all TESS files

In [11]:
n = len(gto)
n

363

#### Batch processing

In [12]:
# Batch processing:
lapse_list = []
median_lapse = None
#for i in range(0, 3):
#for i in range(0, 72): # Record 72 took too long to complete.
#for i in range(73, 176): # The final records were taking much longer to complete.
#for i in range(176, 246): # Aborted at 246 for convenience
#for i in range(246, 250): # Record 250 took too long to complete.
#for i in range(251, 300): # Record 250 took too long to complete.
for i in range(300, len(gto)):
    start_time = time.time()
    # Names:
    karmn = gto.loc[i, 'Karmn'] # KARMENES name
    commn = gto.loc[i, 'Name'] # Common name
    tic_id = str(gto.loc[i, 'TIC_id']) # TESS TIC identifier
    print("Record: %d, started at %s"
          %(i, time.strftime('%d/%m/%Y, %H:%M:%S', time.localtime(start_time))))
    if median_lapse is None:
        print("Previous median lapse time: %s" %median_lapse)
    else:
        print("Previous median lapse time: %.2f seconds" %median_lapse)
    print("Processing %s star (TIC: %s)..." %(karmn, tic_id))
    try:
        # LOAD LC FILE:
        lc_file = gto.loc[i, 'lc_file']
        print("filename: %s" %lc_file)
        lc_lk = tess_lc_load(lc_file)
        
        # GENERATE PERIODOGRAM:
        gls = Gls((lc_lk['time'].value, lc_lk['flux'].value, lc_lk['flux_err'].value),
              fbeg=FBEG, fend=FEND, Pbeg=PBEG, Pend=PEND, ofac=OFAC, hifac=HIFAC, freq=FREQ,
              norm=NORM, ls=LS, fast=FAST, verbose=VERBOSE)
        
        # GENERATE THE FITS FILE:
        # Prepare the fits primary HDU (only header):
        primary_header = fits.Header()
        primary_header['OBJECT'] = (karmn, "KARMENES target name")
        primary_header['NAME'] = (commn, "Object common name")
        primary_header['TIC'] = (tic_id, "Object TESS identifier")
        primary_header['RA_J2000'] = ("00:05:10.89", "Object right ascension (J2000)")
        primary_header['DE_J2000'] = ("+45:47:11.6", "Object declination (J2000)")
        primary_header['SPTYPE'] = ("M1.0 V", "Spectral type")
        primary_header['TEFF_K'] = (3773, "Effective temperature in Kelvin")
        primary_header['LOGG'] = (5.07, "Logarithm of surface gravity")
        primary_header['FEH'] = (-0.04, "Metallicity")
        primary_header['L_LSUN'] = (0.0436229, "Luminosity in Solar luminosities")
        primary_header['R_RSUN'] = (0.48881, "Radius in Solar radii")
        primary_header['M_MSUN'] = (0.4918, "Mass in Solar masses")
        primary_header['D_PC'] = (11.50352803, "Distance in parsec")
        hdu_primary = fits.PrimaryHDU(header=primary_header)

        # Prepare the LC HDU:
        hdu_lc = fits.table_to_hdu(QTable(lc_lk.to_table()))
        hdu_lc.name = "TESS_LC"
        psample = lc_infer_sampling(lc_lk).value
        fsample = 1.0 / psample
        fnyq = 2.0 * fsample
        freq_units = u.d ** (-1)
        hdu_lc.header['OBJECT'] = (karmn, "KARMENES target name")
        hdu_lc.header['PUNIT'] = u.d.to_string(format='fits')
        hdu_lc.header['FUNIT'] = freq_units.to_string(format='fits')
        hdu_lc.header['RVPOINTS'] = (gls.N, "Number of points in the RV curve")
        hdu_lc.header['AVGFLUX'] = (gls._Y, "Average flux of RV curve")
        hdu_lc.header['RMSFLUX'] = (np.sqrt(gls._YY), "Flux RMS of RV curve")
        hdu_lc.header['PSAMPLE'] = (psample, "Inferred cadence in RV curve")
        hdu_lc.header['FSAMPLE'] = (fsample, "Inferred sampling frequency in RV curve")
        hdu_lc.header['FNYQUIST'] = (fnyq, "Calculated Nyquist frequency value")
                
        # Prepare the PG HDU:
        hdu_pg = fits.table_to_hdu(
            QTable(data=[gls.freq, gls.power], names=['freq', 'power'], 
                   units=[1.0 / u.d, (u.electron / u.s) ** 2]))
        hdu_pg.name = "GLS_PG"
        fpoints = len(gls.f)
        fres = (gls.fend - gls.fbeg) / (fpoints - 1)
        hdu_pg.header['OBJECT'] = (karmn, "KARMENES target name")
        hdu_pg.header['FUNIT'] = (freq_units.to_string(format='fits'), "Unit for frequencies")
        hdu_pg.header['PUNIT'] = (u.d.to_string(format='fits'), "Unit for periods")
        hdu_pg.header['PK_FREQ'] = (gls.best['f'], "Frequency of the peak in periodogram")
        hdu_pg.header['PK_POW'] = (gls.pmax, "Power of the peak in periodogram")
        hdu_pg.header['PK_SNR'] = (gls.best['amp'] / gls.rms, "SNR of the peak in periodogram")
        hdu_pg.header['PK_FAP'] = (gls.FAP(Pn=None), "FAP of the peak in periodogram")
        hdu_pg.header['RES_RMS'] = (gls.rms, "RMS of residuals in periodogram")
        hdu_pg.header['FSAMPLE'] = (fsample, "Inferred sampling frequency in TESS curve")
        hdu_pg.header['FNYQUIST'] = (fnyq, "Calculated Nyquist frequency value")
        hdu_pg.header['FPOINTS'] = (fpoints, "Number of points in periodogram")
        hdu_pg.header['FBEG'] = (gls.fbeg, "Start frequency in periodogram")
        hdu_pg.header['FEND'] = (gls.fend, "End frequency in periodogram")
        hdu_pg.header['FRES'] = (fres, "Frequency resolution in periodogram")
        hdu_pg.header['F'] = (gls.best['f'], "Peak best estimate: frequency")
        hdu_pg.header['E_F'] = (gls.best['e_f'], "Peak best estimate: frequency error")
        hdu_pg.header['P'] = (gls.best['P'], "Peak best estimate: period")
        hdu_pg.header['E_P'] = (gls.best['e_P'], "Peak best estimate: period error")
        hdu_pg.header['A'] = (gls.best['amp'], "Peak best estimate: amplitude")
        hdu_pg.header['E_A'] = (gls.best['e_amp'], "Peak best estimate: amplitude error")
        hdu_pg.header['PH'] = (gls.best['ph'], "Peak best estimate: phase")
        hdu_pg.header['E_PH'] = (gls.best['e_ph'], "Peak best estimate: phase error")
        hdu_pg.header['T0'] = (gls.best['T0'], "Peak best estimate: frequency")
        hdu_pg.header['E_T0'] = (gls.best['e_T0'], "Peak best estimate: frequency error")
        hdu_pg.header['OFF'] = (gls.best['offset'], "Peak best estimate: offset")
        hdu_pg.header['E_OFF'] = (gls.best['e_offset'], "Peak best estimate: offset error")
        hdu_pg.header['OFAC'] = (gls.ofac, "Setup: oversampling factor")
        hdu_pg.header['HIFAC'] = (gls.ofac, "Setup: maximum frequency factor")
        hdu_pg.header['NORM'] = (gls.norm, "Setup: normalization type")
        hdu_pg.header['LS'] = (gls.ls, "Setup: conventional Lomb-Scargle calculation")
        hdu_pg.header['FAST'] = (gls.fast, "Setup: fast evaluation, recursive trigonometric")
        
        # Create and save the fits file:
        fits_filename = OUT_PROCESSED_FOLDER + karmn + "_TESS_PG.fits"
        hdul = fits.HDUList([hdu_primary, hdu_lc, hdu_pg])
        hdul.writeto(fits_filename, overwrite=True)
        
        # Delete all the HDUs:
        del hdul
        del hdu_primary
        del hdu_lc
        del hdu_pg

        # FILL IN THE DATA IN THE GTO TABLE:
        gto.loc[i, 'n_TESS'] = gls.N
        gto.loc[i, 'Ps_TESS'] = psample
        gto.loc[i, 'fs_TESS'] = fsample
        gto.loc[i, 'wmean_TESS'] = gls._Y
        gto.loc[i, 'wrms_TESS'] = np.sqrt(gls._YY)
        gto.loc[i, 'info_PG_TESS'] = gls.info(stdout=False)
        gto.loc[i, 'maxP_PG_TESS'] = gls.power.max()
        gto.loc[i, 'maxSNR_PG_TESS'] = gls.best['amp'] / gls.rms
        gto.loc[i, 'rms_PG_TESS'] = gls.rms
        gto.loc[i, 'f_PG_TESS'] = gls.best['f']
        gto.loc[i, 'e_f_PG_TESS'] = gls.best['e_f']
        gto.loc[i, 'Pd_PG_TESS'] = gls.best['P']
        gto.loc[i, 'e_Pd_PG_TESS'] = gls.best['e_P']
        gto.loc[i, 'Ph_PG_TESS'] = 24.0 * gls.best['P']
        gto.loc[i, 'e_Ph_PG_TESS'] = 24.0 * gls.best['e_P']
        gto.loc[i, 'Pm_PG_TESS'] = 24.0 * 60.0 * gls.best['P']
        gto.loc[i, 'e_Pm_PG_TESS'] = 24.0 * 60.0 * gls.best['e_P']
        gto.loc[i, 'A_PG_TESS'] = gls.best['amp']
        gto.loc[i, 'e_A_PG_TESS'] = gls.best['e_amp']
        gto.loc[i, 'ph_PG_TESS'] = gls.best['ph']
        gto.loc[i, 'e_ph_PG_TESS'] = gls.best['e_ph']
        gto.loc[i, 'T0_PG_TESS'] = gls.best['T0']
        gto.loc[i, 'e_T0_PG_TESS'] = gls.best['e_T0']
        gto.loc[i, 'offset_PG_TESS'] = gls.best['offset']
        gto.loc[i, 'e_offset_PG_TESS'] = gls.best['e_offset']
        gto.loc[i, 'FAP_PG_TESS'] = gls.FAP(Pn=None)

        # GENERATE THE FIGURE:
        fig = gls.plot(block=False, period=False,
                       fap=FAP_LEVELS_PLOT, gls=True, data=True, residuals=True)
        figure_title = "TESS LC: %s (TIC %s), P=%.4f [min], FAP=%.4f%%" \
            %(karmn, tic_id, gto.loc[i, 'Pm_PG_TESS'], 100.0 * gto.loc[i, 'FAP_PG_TESS'])
        fig.suptitle(figure_title, fontdict = {'fontsize' : 36})
        fig.tight_layout()
        # Save the figure to disk:
        fig_file = OUT_IMG_FOLDER + karmn + "_TESS_PG.png"
        fig.savefig(fig_file)
        plt.close() # Prevent the figure from showing.

        # SET THE RECORD CALCULATION AS VALID AND STORE THE REULTING FILENAMES
        gto.loc[i, 'valid_PG_TESS'] = 1
        gto.loc[i, 'error_PG_TESS'] = ""
        gto.loc[i, 'fits_file_TESS'] = fits_filename
        gto.loc[i, 'fig_file_TESS'] = fig_file

        # UPDATE THE AVERAGE RECORD PROCESSING TIME:
        lapse = time.time() - start_time
        lapse_list.append(lapse)
        median_lapse = np.nanmedian(lapse_list)
        gto.loc[i, 'elapsed_time_PG_TESS'] = lapse
        
        # SAVE THE UPDATED GTO TABLE TO DISK:
        gto.to_csv(GTO_FILE, sep=',', decimal='.', index=False)
        
        # Report successful execution:
        print("Elapsed time: %.2f seconds" %lapse)
        print("... SUCCESS.")
        
    except Exception as e:
        # Some error happened, establish the record as not valid and record the error:
        error = "--- Some ERROR happened with record #%d, %s star. Error=%s" %(i, karmn, e)
        print(error)
        gto.loc[i, 'valid_PG_TESS'] = 0
        gto.loc[i, 'error_PG_TESS'] = e
        
        # Try to update the record, and save the file:
        try:
            # UPDATE THE AVERAGE RECORD PROCESSING TIME:
            lapse = time.time() - start_time
            lapse_list.append(lapse)
            median_lapse = np.nanmedian(lapse_list)
            gto.loc[i, 'elapsed_time_PG_TESS'] = lapse
            print("Elapsed time: %.2f seconds" %lapse)

            # SAVE THE UPDATED GTO TABLE TO DISK:
            gto.to_csv(GTO_FILE, sep=',', decimal='.', index=False)
        except Exception as e2:
            error_2 = "--- Additional ERROR happened with record #%d, %s star. Error=%s" %(i, karmn, str(e2))
            print(error)
            gto.loc[i, 'error_PG_TESS'] = gto.loc[i, 'error_PG_TESS'] + "/" + str(e2)



Record: 300, started at 29/03/2022, 16:35:40
Previous median lapse time: None
Processing J04406-128 star (TIC: 117799904)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J04406-128/J04406-128_TESS_TIC117799904_S32.fits


c:\users\ciroe\gls.py:642: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


Elapsed time: 80.75 seconds
... SUCCESS.
Record: 301, started at 29/03/2022, 16:37:00
Previous median lapse time: 80.75 seconds
Processing J04376-110 star (TIC: 55997717)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J04376-110/J04376-110_TESS_TIC55997717_S32.fits


c:\users\ciroe\gls.py:642: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


Elapsed time: 456.39 seconds
... SUCCESS.
Record: 302, started at 29/03/2022, 16:44:37
Previous median lapse time: 268.57 seconds
Processing J04376+528 star (TIC: 274545692)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J04376+528/J04376+528_TESS_TIC274545692_S19.fits


...

Elapsed time: 533.30 seconds
... SUCCESS.
Record: 359, started at 29/03/2022, 23:05:56
Previous median lapse time: 472.85 seconds
Processing J00184+440 star (TIC: 440109718)...
filename: nan
--- Some ERROR happened with record #359, J00184+440 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 360, started at 29/03/2022, 23:05:56
Previous median lapse time: 470.78 seconds
Processing J00183+440 star (TIC: 440109725)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J00183+440/J00183+440_TESS_TIC440109725_S17.fits


c:\users\ciroe\gls.py:642: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


Elapsed time: 550.93 seconds
... SUCCESS.
Record: 361, started at 29/03/2022, 23:15:07
Previous median lapse time: 472.85 seconds
Processing J00067-075 star (TIC: 176287658)...
filename: nan
--- Some ERROR happened with record #361, J00067-075 star. Error=Not recognized as a supported data product:
nan
This file may be corrupt due to an interrupted download. Please remove it from your disk and try again.
Elapsed time: 0.00 seconds
Record: 362, started at 29/03/2022, 23:15:07
Previous median lapse time: 470.78 seconds
Processing J00051+457 star (TIC: 439946126)...
filename: ../data/CARMENES_GTO_TESS_lc/lightcurves/J00051+457/J00051+457_TESS_TIC439946126_S17.fits


c:\users\ciroe\gls.py:642: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


Elapsed time: 558.62 seconds
... SUCCESS.


In [ ]:
gto.loc[0:72, ['lc_file', 'error_PG_TESS']]

In [ ]:
gto.loc[gto['lc_file'] == '<ambiguous>', ['Karmn']]

## Review a given file

In [ ]:
object_name = "J23548+385"

In [ ]:
read_hdul = fits.open(OUT_PROCESSED_FOLDER + object_name + "_TESS_PG.fits",
                      mode='update', memmap=False)
read_hdul

In [ ]:
read_hdul.info()

### Retrieve basic information

In [ ]:
read_hdul['PRIMARY'].header

### Retrieve RV curve information

In [ ]:
read_hdul['TESS_LC']

In [ ]:
read_hdul['TESS_LC'].header

In [ ]:
read_hdul['TESS_LC'].data[:10]

In [ ]:
read_rv_table = QTable.read(read_hdul['TESS_LC'])
read_rv_table

### Retrieve periodogram information

In [ ]:
read_hdul['GLS_PG']

In [ ]:
read_hdul['GLS_PG'].header

In [ ]:
read_hdul['GLS_PG'].data[:10]

In [ ]:
read_pg_table = QTable.read(read_hdul['GLS_PG'])
read_pg_table

In [ ]:
read_pg_table['freq'][:-1]

In [ ]:
diff_frecs = read_pg_table['freq'][:-1] - read_pg_table['freq'][1:]
diff_frecs

### Delete read information

In [ ]:
read_hdul.close()

In [ ]:
del read_hdul

# Summary

**OBSERVATIONS AND CONCLUSIONS:**
- We have completed the basic GLS periodogram calculation for the available TESS light curves for GTO objetcs, and stored the results.
- Some of the objects yielded errors, and some others yielded warnings. Those objects should be analized individually.